# Panoptic Quality : supplementary materials

This notebook supplements the paper:

A. Foucart, O. Debeir, C. Decaestecker. "Why Panoptic Quality should be avoided as a metric for assessing cell nuclei segmentation and classification in digital pathology", 2022.

## Downloading and pre-processing the data

To reproduce the results from this notebook, you need to:

* Download the multi-rater evaluation dataset [CSV files]([Drive with CSV files](https://drive.google.com/drive/folders/16P04eKeX3n5oRx3MxODISXaRTbHU22U0))) from the [NuCLS challenge](https://sites.google.com/view/nucls/multi-rater?authuser=0) and put them in the "./nucls_csv" directory.
* Download the MoNuSAC [test set annotations & teams predictions](https://monusac-2020.grand-challenge.org/Data/) and set them in the "./monusac_annotations" and "./monusac_teams" directories.
* Generate "nary masks" from the MoNuSAC annotations & teams predictions for easier processing afterwards using the code below. This operation may take a while. Requires [Openslide](https://openslide.org/). 

In [ ]:
from monusac import generate_nary_masks_from_annotations, generate_nary_masks_from_teams

generate_nary_masks_from_annotations("./monusac_annotations")
generate_nary_masks_from_teams("./monusac_teams")

## 1. NuCLS

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
from nucls import load_annotations, match_experts

NUCLS_PATH = "./nucls_csv"

annotations, all_slides = load_annotations(NUCLS_PATH)
matches = match_experts(annotations, all_slides)

In [ ]:
bins = [0, 500, 1000, 1500, 2000, 5000]
binned = [[] for _ in bins[1:]]

for match in matches:
    for i,b in enumerate(bins[1:]):
        if match.area < b:
            binned[i].append(match.iou)
            break
            
plt.figure(figsize=(15,6))
plt.boxplot(binned)
plt.xticks(range(1, 6), [f'[{bins[i-1]},{bins[i]}[' for i in range(1, len(bins))])
plt.xlabel('Area (px)', fontsize=12)
plt.ylabel('IoU', fontsize=12)
plt.show()

In [ ]:
from skimage.measure import find_contours
from skimage.io import imread
import os
from metrics import compute_iou, compute_hd

path_single = "./example_single_cell"

rgb = imread(os.path.join(path_single, 'rgb.png'))
gt = imread(os.path.join(path_single, 'gt.png'))>0
seg_1 = imread(os.path.join(path_single, 'seg_1.png'))>0
seg_2 = imread(os.path.join(path_single, 'seg_2.png'))>0
seg_3 = imread(os.path.join(path_single, 'seg_3.png'))>0
seg_4 = imread(os.path.join(path_single, 'seg_4.png'))>0

ref_contours = find_contours(gt)[0]

plt.figure(figsize=(15,5))
plt.subplot(1, 5, 1)
plt.imshow(rgb)
for i,mask in enumerate([seg_1, seg_2, seg_3, seg_4]):
    contours = find_contours(mask)[0]
    plt.subplot(1, 5, i+2)
    plt.imshow(rgb)
    plt.plot(ref_contours[:, 1], ref_contours[:, 0], 'b-')
    plt.plot(contours[:, 1], contours[:, 0], 'k--')
    plt.title(f'IoU={compute_iou(gt, mask):.2f}')
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

x = np.array([match.iou for match in matches])
y = np.array([match.hd for match in matches])

coeffs = np.polyfit(np.log(x), y, deg=1)
y_pred = sum([c*(x**(len(coeffs)-i-1)) for i,c in enumerate(coeffs)])

x_ = np.linspace(0.01, 1., 100)
plt.figure(figsize=(15,7))
plt.plot(x, y, 'b+')
plt.plot([0, 1], [3, 3], 'k-')
plt.plot(x_, sum([c*(np.log(x_)**(len(coeffs)-i-1)) for i,c in enumerate(coeffs)]), 'r-')
plt.text(0, -0.5, 'HD=3')
plt.xlabel('IoU', fontsize=14)
plt.ylabel('HD', fontsize=14)
plt.gca().tick_params(axis='both', which='major', labelsize=12)
plt.show()

## 2. MoNuSAC experiments

In [ ]:
from monusac import get_all_nuclei

MONUSAC_ANNOTATIONS_PATH = "./monusac_annotations/"
MONUSAC_TEAMS_PATH = "./monusac_teams/"

nuclei = get_all_nuclei(MONUSAC_ANNOTATIONS_PATH)

Experiment on all nuclei: eroded, dilated and shifted

In [ ]:
from skimage.morphology import erosion, dilation, disk
from tqdm import tqdm
import numpy as np
from metrics import compute_iou, compute_hd

se = disk(1)

ious = {}

for cl_type, cl_nuclei in nuclei.items():
    ious[cl_type] = []
    for nucleus in tqdm(cl_nuclei):
        eroded = erosion(nucleus.mask, se)
        dilated = dilation(nucleus.mask, se)
        shifted = np.zeros_like(nucleus.mask)
        shifted[1:] = nucleus.mask[:-1]
        ious[cl_type].append((compute_iou(nucleus.mask, eroded),
                              compute_iou(nucleus.mask, dilated),
                              compute_iou(nucleus.mask, shifted),
                              nucleus.area))

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

plt.figure(figsize=(15,7))
plt.boxplot([[iou[3] for iou in ious[cl]] for cl in ious])
plt.xticks([1, 2, 3, 4], [cl for cl in ious])
plt.ylabel('Area (px)', fontsize=15)
plt.gca().tick_params(axis='both', which='major', labelsize=15)
plt.show()

In [ ]:
print("Median and interquartile range:")
for cl in ious:
    areas = [iou[3] for iou in ious[cl]]
    areas.sort()
    q25 = int(0.25*len(areas))
    q75 = int(0.75*len(areas))
    print(f"{cl}: {np.median(areas)} [{areas[q25]} - {areas[q75]}]")
    

In [ ]:
plt.figure(figsize=(20,5))
plt.boxplot([[iou[0] for iou in ious[cl]] for cl in ious], positions=[1, 5, 9, 13], showfliers=False)
plt.boxplot([[iou[1] for iou in ious[cl]] for cl in ious], positions=[2, 6, 10, 14], showfliers=False)
plt.boxplot([[iou[2] for iou in ious[cl]] for cl in ious], positions=[3, 7, 11, 15], showfliers=False)
plt.xticks([1, 2, 3, 5, 6, 7, 9, 10, 11, 13, 14, 15], ['Ee', 'Ed', 'Es', 'Le', 'Ld', 'Ls', 'Ne', 'Nd', 'Ns', 'Me', 'Md', 'Ms'])
plt.ylabel('IoU', fontsize=14)
plt.gca().tick_params(axis='both', which='major', labelsize=14)
plt.show()

In [ ]:
print("Median and interquartile range:")
for cl in ious:
    e = [iou[0] for iou in ious[cl]]
    d = [iou[1] for iou in ious[cl]]
    s = [iou[2] for iou in ious[cl]]
    e.sort()
    d.sort()
    s.sort()
    
    q25 = int(0.25*len(e))
    q75 = int(0.75*len(e))
    
    print(cl)
    print(f"Eroded: {np.median(e):.2f} [{e[q25]:.2f} - {e[q75]:.2f}]")
    print(f"Dilated: {np.median(d):.2f} [{d[q25]:.2f} - {d[q75]:.2f}]")
    print(f"Shifted: {np.median(s):.2f} [{s[q25]:.2f} - {s[q75]:.2f}]")

Simulating what would happen at 20x instead of 40x

In [ ]:
from skimage.morphology import erosion, dilation, disk
from skimage.transform import downscale_local_mean
from tqdm import tqdm
import numpy as np
from metrics import compute_iou, compute_hd

se = disk(1)

ious = {}

for cl_type, cl_nuclei in nuclei.items():
    ious[cl_type] = []
    for nucleus in tqdm(cl_nuclei):
        mask = downscale_local_mean(nucleus.mask, (2, 2))>0
        eroded = erosion(mask, se)
        dilated = dilation(mask, se)
        shifted = np.zeros_like(mask)
        shifted[1:] = mask[:-1]
        ious[cl_type].append((compute_iou(mask, eroded),
                              compute_iou(mask, dilated),
                              compute_iou(mask, shifted),
                              mask.sum()))

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

plt.figure(figsize=(10,5))
plt.boxplot([[iou[3] for iou in ious[cl]] for cl in ious])
plt.xticks([1, 2, 3, 4], [cl for cl in ious])
plt.ylabel('Area (px)')
plt.show()

In [ ]:
print("Median and interquartile range:")
for cl in ious:
    areas = [iou[3] for iou in ious[cl]]
    areas.sort()
    q25 = int(0.25*len(areas))
    q75 = int(0.75*len(areas))
    print(f"{cl}: {np.median(areas)} [{areas[q25]} - {areas[q75]}]")

In [ ]:
plt.figure(figsize=(20,5))
plt.boxplot([[iou[0] for iou in ious[cl]] for cl in ious], positions=[1, 5, 9, 13], showfliers=False)
plt.boxplot([[iou[1] for iou in ious[cl]] for cl in ious], positions=[2, 6, 10, 14], showfliers=False)
plt.boxplot([[iou[2] for iou in ious[cl]] for cl in ious], positions=[3, 7, 11, 15], showfliers=False)
plt.xticks([1, 2, 3, 5, 6, 7, 9, 10, 11, 13, 14, 15], ['Ee', 'Ed', 'Es', 'Le', 'Ld', 'Ls', 'Ne', 'Nd', 'Ns', 'Me', 'Md', 'Ms'])
plt.ylabel('IoU')
plt.show()

In [ ]:
print("Median and interquartile range:")
for cl in ious:
    e = [iou[0] for iou in ious[cl]]
    d = [iou[1] for iou in ious[cl]]
    s = [iou[2] for iou in ious[cl]]
    e.sort()
    d.sort()
    s.sort()
    
    q25 = int(0.25*len(e))
    q75 = int(0.75*len(e))
    
    print(cl)
    print(f"Eroded: {np.median(e):.2f} [{e[q25]:.2f} - {e[q75]:.2f}]")
    print(f"Dilated: {np.median(d):.2f} [{d[q25]:.2f} - {d[q75]:.2f}]")
    print(f"Shifted: {np.median(s):.2f} [{s[q25]:.2f} - {s[q75]:.2f}]")

Checking the results of the different team per nucleus:

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
from monusac import show_nucleus_team_comparison

# Checking results for a single nucleus:
to_show = [
    ['Epithelial', 2],
    ['Lymphocyte', 15],
    ['Neutrophil', 5],
    ['Macrophage', 28]
]
# Epithelial - 2 -> IoU < 0.5 for very good segmentation
# Lymphocyte - 15 -> bad IoU for good segmentation + better IoU for worse shape
# Neutrophil - 5 -> nearly identical segmentation, large range of scores
# Macrophage - 28 -> class mismatch means worse PQ than missed detection

for cl, idn in to_show:
    show_nucleus_team_comparison(MONUSAC_TEAMS_PATH, nuclei, cl, idn)

In [ ]:
from monusac import show_matches_under_threshold

image_file = "TCGA-2Z-A9JG-01Z-00-DX1\TCGA-2Z-A9JG-01Z-00-DX1_4"
idt = 3

show_matches_under_threshold(image_file, idt, MONUSAC_TEAMS_PATH, MONUSAC_ANNOTATIONS_PATH)

Full PQ computation from an example with many nuclei:

In [ ]:
from monusac import find_image_with_most_nuclei, match_best_iou

n, path = find_image_with_most_nuclei(nuclei)
print(f"Most nuclei in image: {path} ({n=})")

In [ ]:
from skimage.io import imread
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np

im = imread(f"{path}.tif")
nary = np.load(f"{path}_nary.npy")
# remove "ambiguous" regions
mask_ambiguous = nary[..., 1]==5
nary[mask_ambiguous, 1] = 0
nary[mask_ambiguous, 0] = 0
im[mask_ambiguous, 3] = 0

plt.figure(figsize=(20,10))
plt.imshow(im)
plt.contour(nary[..., 1]==1, linewidths=0.5, alpha=1, colors=['yellow'])
plt.contour(nary[..., 1]==2, linewidths=0.5, alpha=1, colors=['blue'])
plt.show()

Creating synthetic "predictions" by either removing full objects ("detection errors"), or eroding the objects ("segmentation errors"):

In [ ]:
from tqdm import tqdm

pred_d = np.zeros_like(nary)

# randomly remove 40% of the objects
np.random.seed(5)
r = np.random.random((len(np.unique(nary[..., 0]))))
newi = 0
for i in tqdm(np.unique(nary[..., 0])):
    if i == 0:
        continue
    if r[i] > 0.4:
        newi += 1
        pred_d[nary[..., 0]==i, 0] = newi
        pred_d[nary[..., 0]==i, 1] = nary[nary[..., 0]==i, 1].max()

In [ ]:
from skimage.morphology import disk, erosion
from tqdm import tqdm

pred_s = np.zeros_like(nary)

# Remove small part of each object
rs = []
for i in tqdm(np.unique(nary[..., 0])):
    if i == 0:
        continue
    area = (nary[..., 0]==i).sum()
    r = int(np.round(np.sqrt(area/np.pi)*0.15))
    rs.append(r)
    mask = erosion(nary[..., 0]==i, disk(r))
    pred_s[mask, 0] = i
    pred_s[mask, 1] = nary[nary[..., 0]==i, 1].max()

In [ ]:
print(f"Mean erosion = {np.mean(rs)}px, median erosion = {np.median(rs)}px")
print(f"Count of objects with erosion of ... px:")
rs = np.array(rs)
for r in np.unique(rs):
    print(f"{r}px: {(rs==r).sum()}")

In [ ]:
matches_d = match_best_iou(nary, pred_d)
matches_s = match_best_iou(nary, pred_s)

In [ ]:
from metrics import compute_panoptic_quality
pq_d = compute_panoptic_quality(matches_d)
pq_s = compute_panoptic_quality(matches_s)

In [ ]:
print(f"Kept {100*(len(np.unique(pred_d[..., 0]))-1)/(len(np.unique(nary[..., 0]))-1):.1f}% of the objects for detection errors")
print(f"Kept {100*(pred_s[..., 0]>0).sum()/(nary[..., 0]>0).sum():.1f}% of the objects area for segmentation errors")

Results for the "detection errors" predictions:

In [ ]:
print(f"PQ = {pq_d['PQ']:.2f}")
print(f"RQc = {pq_d['RQc']}")
print(f"SQc = {pq_d['SQc']}")

Results for the "segmentation errors" predictions

In [ ]:
print(f"PQ = {pq_s['PQ']:.2f}")
print(f"RQc = {pq_s['RQc']}")
print(f"SQc = {pq_s['SQc']}")

Visual comparison of the results

In [ ]:
import matplotlib.patches as patches

plt.figure(figsize=(20,20))
sq = patches.Rectangle((1100, 1300), 200, 200, linewidth=2, edgecolor='k', facecolor='none', zorder=100)
ax = plt.subplot(1, 3, 1)
plt.imshow(im)
plt.contour(pred_d[..., 1]==1, linewidths=0.5, alpha=1, colors=['yellow'])
plt.contour(pred_d[..., 1]==2, linewidths=0.5, alpha=1, colors=['blue'])
ax.add_patch(sq)
plt.title(f"PQ={pq_d['PQ']:.2f} (detection errors)")
sq2 = patches.Rectangle((1100, 1300), 200, 200, linewidth=2, edgecolor='k', facecolor='none', zorder=100)
ax = plt.subplot(1, 3, 2)
plt.imshow(im)
plt.contour(nary[..., 1]==1, linewidths=0.5, alpha=1, colors=['yellow'])
plt.contour(nary[..., 1]==2, linewidths=0.5, alpha=1, colors=['blue'])
plt.title('Reference')
ax.add_patch(sq2)
sq3 = patches.Rectangle((1100, 1300), 200, 200, linewidth=2, edgecolor='k', facecolor='none', zorder=100)
ax = plt.subplot(1, 3, 3)
plt.imshow(im)
plt.contour(pred_s[..., 1]==1, linewidths=0.5, alpha=1, colors=['yellow'])
plt.contour(pred_s[..., 1]==2, linewidths=0.5, alpha=1, colors=['blue'])
plt.title(f"PQ={pq_s['PQ']:.2f} (segmentation errors)")
ax.add_patch(sq3)
plt.show()


Details

In [ ]:
detail_d = pred_d[1300:1500,1100:1300]
detail_n = nary[1300:1500,1100:1300]
detail_s = pred_s[1300:1500,1100:1300]
detail = im[1300:1500,1100:1300]

plt.figure(figsize=(20,20))
plt.subplot(1, 3, 1)
plt.imshow(detail)
plt.contour(detail_d[..., 1]==1, linewidths=0.5, alpha=1, colors=['yellow'])
plt.contour(detail_d[..., 1]==2, linewidths=0.5, alpha=1, colors=['blue'])
# plt.title(f"PQ={pq_d['PQ']:.2f} (detection errors)")
plt.axis('off')
plt.subplot(1, 3, 2)
plt.imshow(detail)
plt.contour(detail_n[..., 1]==1, linewidths=0.5, alpha=1, colors=['yellow'])
plt.contour(detail_n[..., 1]==2, linewidths=0.5, alpha=1, colors=['blue'])
plt.axis('off')
# plt.title('Reference')
plt.subplot(1, 3, 3)
plt.imshow(detail)
plt.contour(detail_s[..., 1]==1, linewidths=0.5, alpha=1, colors=['yellow'])
plt.contour(detail_s[..., 1]==2, linewidths=0.5, alpha=1, colors=['blue'])
plt.axis('off')
# plt.title(f"PQ={pq_s['PQ']:.2f} (segmentation errors)")
plt.show()